In [3]:
import os
from os import kill
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
import time
import pandas as pd
import numpy as np



In [27]:
driver = webdriver.Chrome("C:/chromedriver")  #브라우저 켜기
driver.get('https://pubmed.ncbi.nlm.nih.gov/') 
toAdvance = driver.find_element_by_css_selector('a.search-input-link')
toAdvance.click()

# title과 abstract으로 설정
title_abstrac = driver.find_element_by_css_selector('select#field-selector')
title_abstrac.click()
opt = driver.find_element_by_css_selector('#field-selector > option:nth-child(39)')
opt.click()

# Input Keywords
search = driver.find_element_by_css_selector('input#id_term')
search.click()
search.send_keys('(bristol stool scale[Title/Abstract]) AND ((gastrointestinal[Title/Abstract]) OR (disease[Title/Abstract]) OR (diagnosis[Title/Abstract]) OR (prediction[Title/Abstract]) OR (clinical[Title/Abstract]) OR (implication[Title/Abstract]))')
search.send_keys(Keys.ENTER)


searchBtn = driver.find_element_by_css_selector('#search-form > div > div > div.query-box-section-wrapper > div.button-wrapper > button')
searchBtn.click()

title_arr = []
author_arr = []
journal_arr = []
link_arr = []
abstract_arr = []

titles = driver.find_elements_by_css_selector('a.docsum-title')

cnt = 0

while True:
    result = driver.find_element_by_css_selector('div.results-amount > span.value').text
    titles = driver.find_elements_by_css_selector('a.docsum-title')
    num = driver.find_element_by_css_selector('#search-results > section > div.search-results-chunks > div > article:nth-child(2) > div.item-selector-wrap.selectors-and-actions.first-selector > label > span').text
    result = driver.find_element_by_css_selector('div.results-amount > span.value').text
    # print("lenght: ", len(titles))
    # print("num:", num)
    if len(titles) <= 10:
        for i in range(len(titles)):
            titles2 = driver.find_elements_by_css_selector('a.docsum-title')
            titles2[i].click()

            #title
            title = driver.find_element_by_css_selector('h1.heading-title')
            title_arr.append(title.text)

            #authors
            authors = driver.find_element_by_css_selector('#full-view-heading > div.inline-authors > div > div > span:nth-child(1) > a')
            author_arr.append(authors.text)
            
            # journals
            journal = driver.find_element_by_css_selector('#full-view-heading > div.article-citation > div > div > button') 
            journal_arr.append(journal.text)

            # abstract
            try:
                abstract = driver.find_element_by_css_selector('div.abstract-content')
            except:
                abstract = None
            else:
                abstract_arr.append(abstract.text)

            #link
            try:
                link = driver.find_elements_by_css_selector('a.id-link')
                href= link[-1].get_attribute('href')
            
            except IndexError:
                href = None
            link_arr.append(href)

            driver.back()
            cnt += 1
            
            print(f"Article Count: {cnt}, Proceeding rate: {round(cnt/int(result) * 100, 2)}%")
            if i == 9:
                ## Show More
                more = driver.find_element_by_css_selector('#search-results > section > div.search-results-paginator.next-results-paginator.has-nav > button > span')
                more.click()
                time.sleep(1.2)
    if cnt == result:
        break
    
    elif len(titles) > 10:
        titles2 = driver.find_elements_by_css_selector('a.docsum-title')
        titles2[10].click()
        driver.back()
    
    




Article Count: 1, Proceeding rate: 0.57%
Article Count: 2, Proceeding rate: 1.14%
Article Count: 3, Proceeding rate: 1.7%
Article Count: 4, Proceeding rate: 2.27%
Article Count: 5, Proceeding rate: 2.84%
Article Count: 6, Proceeding rate: 3.41%
Article Count: 7, Proceeding rate: 3.98%
Article Count: 8, Proceeding rate: 4.55%
Article Count: 9, Proceeding rate: 5.11%
Article Count: 10, Proceeding rate: 5.68%
Article Count: 11, Proceeding rate: 6.25%
Article Count: 12, Proceeding rate: 6.82%
Article Count: 13, Proceeding rate: 7.39%
Article Count: 14, Proceeding rate: 7.95%
Article Count: 15, Proceeding rate: 8.52%
Article Count: 16, Proceeding rate: 9.09%
Article Count: 17, Proceeding rate: 9.66%
Article Count: 18, Proceeding rate: 10.23%
Article Count: 19, Proceeding rate: 10.8%
Article Count: 20, Proceeding rate: 11.36%
Article Count: 21, Proceeding rate: 11.93%
Article Count: 22, Proceeding rate: 12.5%
Article Count: 23, Proceeding rate: 13.07%
Article Count: 24, Proceeding rate: 13.6

KeyboardInterrupt: 

In [31]:
bristol_df = pd.DataFrame(np.c_[title_arr, journal_arr, author_arr, link_arr, abstract_arr], columns=['title','journal','authors','links','abstract'])

In [34]:
bristol_df['keyword']=None

In [46]:
keywords= ['predict','diagnos','implication','Bristol']

In [47]:
'Several' in bristol_df['abstract'][1]

True

In [48]:
keyword_arr= {}
for i in range(bristol_df.shape[1]):
    for j in keywords:
        if j in bristol_df['abstract'][i]:
            keyword_arr[i] = j
        

In [49]:
keyword_arr

{0: 'Bristol',
 1: 'Bristol',
 2: 'Bristol',
 3: 'Bristol',
 4: 'Bristol',
 5: 'Bristol'}

In [13]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://pubmed.ncbi.nlm.nih.gov/27492648/"
response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    abstract = soup.select_one('div.abstract-content').text
    print(abstract)

In [19]:
url = "https://pubmed.ncbi.nlm.nih.gov/?term=bristol+stool"
response = requests.get(url)

html = response.text
soup = BeautifulSoup(html, 'html.parser')
title = soup.select("div.docsum-content > a.docsum-title")
title_arr = []

for i in range(len(title)):
    title_arr.append(title[i].text)
print(len(title_arr))

10
